## Generacion de Datos Historicos Coomproriente


In [ ]:
import pandas as pd

# Leer todas las hojas
sheets = pd.read_excel('Lista de precios coomproriente.xlsx', sheet_name=None)

# Combinar todas las hojas en un solo DataFrame, agregando el nombre de la hoja como columna
df_all = pd.concat(
    [df.assign(PRODUCTO=sheet_name) for sheet_name, df in sheets.items()],
    ignore_index=True
)

df_all.info()
# Guardar como CSV en Google Colab
df_all.to_csv('/content/precios_todos_los_productos.csv', index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   PRODUCTO  1240 non-null   object        
 1   PRECIO    1240 non-null   object        
 2   CANTIDAD  1240 non-null   object        
 3   FECHA     1240 non-null   datetime64[ns]
 4   ORIGEN    1240 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 48.6+ KB


In [ ]:
df_all['FECHA'].unique()

(23,)

In [ ]:
productos=df_all['PRODUCTO'].unique()

## Generar datos sinteticos de cada uno de los productos desde enero hasta la ultima fecha


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Convertir PRECIO a numérico
df_all['PRECIO'] = pd.to_numeric(df_all['PRECIO'], errors='coerce')
df_all['FECHA'] = pd.to_datetime(df_all['FECHA'], errors='coerce')

np.random.seed(42)

resultados = []

for producto in df_all['PRODUCTO'].unique():
    df_producto = df_all[df_all['PRODUCTO'] == producto].copy()
    df_producto = df_producto.dropna(subset=['PRECIO', 'FECHA'])
    if df_producto.empty:
        continue

    # Tomar la cantidad más frecuente o el primer valor no nulo
    cantidad = df_producto['CANTIDAD'].mode().iloc[0] if not df_producto['CANTIDAD'].mode().empty else df_producto['CANTIDAD'].iloc[0]

    # Agrupar por fecha y promediar precios si hay datos repetidos
    df_producto = df_producto.groupby('FECHA', as_index=False)['PRECIO'].mean()
    df_producto = df_producto.sort_values('FECHA').reset_index(drop=True)

    # Agregar índice para regresión
    df_producto['DIA'] = np.arange(len(df_producto))

    # Regresión lineal precio vs día
    X = df_producto['DIA'].values.reshape(-1,1)
    modelo_precio = LinearRegression().fit(X, df_producto['PRECIO'].values)

    # Generar 20 días hacia atrás con regresión
    dias_pasado = np.arange(-20, 0).reshape(-1,1)
    precios_pasado = modelo_precio.predict(dias_pasado)

    primer_dia = df_producto['FECHA'].min()
    fechas_pasado = [primer_dia - pd.Timedelta(days=i) for i in range(20, 0, -1)]

    df_pasado = pd.DataFrame({
        'FECHA': fechas_pasado,
        'PRECIO': precios_pasado.round(2)
    })

    # Unir datos pasados y reales
    df_completo = pd.concat([df_pasado, df_producto[['FECHA','PRECIO']]], ignore_index=True).sort_values('FECHA').reset_index(drop=True)

    # Generar rango total de 50 días
    total_dias = 50
    fecha_final = df_completo['FECHA'].min() + pd.Timedelta(days=total_dias-1)
    fechas_totales = pd.date_range(start=df_completo['FECHA'].min(), end=fecha_final, freq='D')

    # Reindexar y interpolar precios faltantes
    df_reindex = df_completo.set_index('FECHA').reindex(fechas_totales)
    df_reindex['PRECIO'] = df_reindex['PRECIO'].interpolate(method='linear').ffill().bfill()

    # Agregar ruido al precio para datos sintéticos
    ruido_precio = np.random.normal(loc=0, scale=2, size=len(df_reindex))
    df_reindex['PRECIO_SINTETICO'] = (df_reindex['PRECIO'] + ruido_precio).round(2)

    # Cantidad fija (string)
    df_reindex['CANTIDAD'] = cantidad

    df_reindex['PRODUCTO'] = producto
    df_reindex = df_reindex.reset_index().rename(columns={'index': 'FECHA'})

    df_final = df_reindex[['FECHA','PRODUCTO','PRECIO_SINTETICO','CANTIDAD']].iloc[:total_dias]

    resultados.append(df_final)

# Concatenar todos los productos
df_sintetico_final = pd.concat(resultados, ignore_index=True)

# Guardar CSV
df_sintetico_final.to_csv('/content/precios_cantidad_sinteticos_fijos.csv', index=False)

print(df_sintetico_final.head())
print(df_sintetico_final.info())


       FECHA            PRODUCTO  PRECIO_SINTETICO           CANTIDAD
0 2025-02-28  Aguacate papelillo              9.81  Kilogramo - 50 Kg
1 2025-03-01  Aguacate papelillo              8.58  Kilogramo - 50 Kg
2 2025-03-02  Aguacate papelillo             10.20  Kilogramo - 50 Kg
3 2025-03-03  Aguacate papelillo             11.98  Kilogramo - 50 Kg
4 2025-03-04  Aguacate papelillo              8.50  Kilogramo - 50 Kg
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3100 entries, 0 to 3099
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   FECHA             3100 non-null   datetime64[ns]
 1   PRODUCTO          3100 non-null   object        
 2   PRECIO_SINTETICO  3100 non-null   float64       
 3   CANTIDAD          3100 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 97.0+ KB
None


## Prediccion de precios para 7 dias

In [ ]:
from prophet import Prophet
import pandas as pd
import numpy as np

# Prepara df_all igual que antes: FECHA datetime, PRECIO numérico, etc.

resultados_pred = []

for producto in df_all['PRODUCTO'].unique():
    df_producto = df_all[df_all['PRODUCTO'] == producto].copy()
    df_producto = df_producto.dropna(subset=['PRECIO', 'FECHA'])
    if len(df_producto) < 5:  # si hay pocos datos, salta
        continue

    # Prophet requiere columnas 'ds' y 'y'
    df_producto = df_producto.groupby('FECHA', as_index=False)['PRECIO'].mean()
    df_producto.rename(columns={'FECHA': 'ds', 'PRECIO': 'y'}, inplace=True)

    model = Prophet(daily_seasonality=True)
    model.fit(df_producto)

    # Crear dataframe futuro para 30 días
    future = model.make_future_dataframe(periods=7)

    forecast = model.predict(future)

    # Extraer fechas y predicción
    df_forecast = forecast[['ds', 'yhat']].tail(7)
    df_forecast['PRODUCTO'] = producto

    # Para mantener el formato
    df_forecast.rename(columns={'ds': 'FECHA', 'yhat': 'PRECIO_PREDICTO'}, inplace=True)

    resultados_pred.append(df_forecast)

df_pred_final = pd.concat(resultados_pred, ignore_index=True)

print(df_pred_final.head())


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 12.
DEBUG:cmdstanpy:input tempfile: /tmp/tmphm5tr5q9/t6ewnku5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmphm5tr5q9/9x25mkv0.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=3158', 'data', 'file=/tmp/tmphm5tr5q9/t6ewnku5.json', 'init=/tmp/tmphm5tr5q9/9x25mkv0.json', 'output', 'file=/tmp/tmphm5tr5q9/prophet_modelcuj8mnmb/prophet_model-20250515163610.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
16:36:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
16:36:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonality. Run prophet with yearl

       FECHA  PRECIO_PREDICTO            PRODUCTO
0 2025-05-09         7.871005  Aguacate papelillo
1 2025-05-10         7.882246  Aguacate papelillo
2 2025-05-11         7.893489  Aguacate papelillo
3 2025-05-12        10.109632  Aguacate papelillo
4 2025-05-13         7.915973  Aguacate papelillo


In [ ]:
df_pred_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   FECHA            427 non-null    datetime64[ns]
 1   PRECIO_PREDICTO  427 non-null    float64       
 2   PRODUCTO         427 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 10.1+ KB


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot()

In [ ]:
df_pred_final.to_csv('/content/precios_cantidad_predichos_prophet.csv', index=False)

